In [26]:
import math
import scipy
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.externals import joblib
import cPickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score

In [4]:
def decodeArray(text):
    try:
        return text.decode('utf-8','replace')
    except:
        return 0

In [29]:
#Reading the jsonfile 
import simplejson as json
f =open("reviews_Grocery_and_Gourmet_Food.json",mode='r')
L = f.readlines()
f.close()
review_list = []
rating = []
reviewer_id = []
product_id = []
for l in L:
    jj = json.loads(l)
    if (len(jj['reviewText'].split(" "))>12):
        review_list += [jj['reviewText']]
        rating += [jj['overall']]
        reviewer_id += [jj['reviewerID']]
        product_id += [jj['asin']]
        

d={}
d['TEXT'] = review_list
d['RATING'] = rating
d['reviewerID'] = reviewer_id
d['product_id'] = product_id

revdf = pd.DataFrame(d)

In [30]:
type(revdf)

pandas.core.frame.DataFrame

In [ ]:
data = pd.read_csv("rev_data_small.csv", sep='\t')
data = data[len(data.REVIEWTEXT) >12]

data[(data.FOOD>4) )].count()
data[data.FOOD<3].count()
data[data.FOOD<3].REVIEWTEXT
good_reviews = data[(data.FOOD>4) ]
good_reviews = good_reviews.reset_index().ix[0:100000]


bad_reviews = data[(data.FOOD<3)]
bad_reviews = bad_reviews.reset_index().ix[0:100000]

good_reviews['target'] = 1
bad_reviews['target'] = 0
all_reviews = good_reviews.append(bad_food_reviews)
 

## Hyper Parameter search code:

<pre><code>

from __future__ import print_function
import pandas as pd

from pprint import pprint
from time import time
import logging

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


###############################################################################
# define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression()),
])


parameters = {
    'vect__max_df': (0.9, 1.0),
    'vect__stop_words': (None, 'english'),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 2), (1, 3)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__penalty': ('l2', 'l1')
    #'clf__n_iter': (10, 50, 80),
}

all_reviews = pd.read_csv('Train_rev.csv')

def decodeArray(text):
    try:
        return text.decode('utf-8','replace')
    except:
        return 0


X = all_reviews.TEXT.apply(decodeArray).values
y = all_reviews.target.values

print(y)



if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=2)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(X, y)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    print(grid_search.best_estimator_.predict_proba(["the food was amazing"]))
    print(grid_search.best_estimator_.predict_proba(["the food was delicious"]))
    print(grid_search.best_estimator_.predict_proba(["the food was not delicious at all"]))
    
 
 
</code></pre>


## this was the result of hyperparameter search:

* Best score: 0.931    
* Best parameters set:
- clf__penalty: 'l1'
- vect__max_df: 0.9
- vect__ngram_range: (1, 2)
- vect__stop_words: None

In [6]:
def decodeArray(text):
    try:
        return text.decode('utf-8','replace')
    except:
        return 0

In [22]:
LR_Vectorizer=TfidfVectorizer(analyzer='word', binary=False,decode_error='strict',
                       encoding='utf-8',lowercase=True,
                       max_df=0.9,max_features=None,min_df=1,ngram_range=(1, 2),norm='l1',
                       preprocessor=None,smooth_idf=False,stop_words=None,strip_accents=None,
                       sublinear_tf=False,token_pattern='(?u)\\b\\w\\w+\\b',tokenizer=None,use_idf=False,vocabulary=None)

LR_clf=linear_model.LogisticRegression(penalty='l1')
LR_pipeline=Pipeline([('vect', LR_Vectorizer),('clf', LR_clf)])

In [8]:
all_food_reviews = pd.read_csv('Train_rev.csv')

In [23]:
X = all_food_reviews['TEXT']

In [24]:
y = all_food_reviews['target']

In [25]:
cross_val_score(LR_pipeline, X, y)

array([ 0.89168596,  0.88843879,  0.88699203])

In [13]:
LR_pipeline.fit(X, y)

Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<type 'numpy.int64'>, encoding='utf-8', input=u'content',
        lowercase=True, max_df=0.9, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l1', preprocessor=None, smooth_idf=False,
 ...',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))])

In [14]:
LR_pipeline.predict_proba(["This chili powder adds a wonderful spiciness and smokiness to dishes. "])

array([[ 0.00433934,  0.99566066]])

In [15]:
LR_pipeline.predict_proba(["Warning, however. Ghost peppers have a spiciness that increases after the first taste, so adjust accordingly."])

array([[ 0.34965657,  0.65034343]])

In [16]:
LR_pipeline.predict_proba(["No sugar, no GMO garbage, no fillers that come with store bought extracts."])

array([[ 0.76197423,  0.23802577]])

In [17]:
LR_pipeline.predict_proba(["This stuff is just amazing."])

array([[ 0.00383323,  0.99616677]])

In [18]:
LR_pipeline.predict_proba(["I use it in everything from baking to cooking and even as suggested in my coffee which is saying a lot because I normally do not care for flavored coffee!"])

array([[ 0.15018488,  0.84981512]])

In [19]:
LR_pipeline.predict_proba(["You cannot go wrong with this."])

array([[ 0.05746628,  0.94253372]])

In [20]:
LR_pipeline.predict_proba(["I've ordered from this merchant before, customer satisfaction is their priority and service was quick, shipped right out with tracking even! I'll be buying from GLS Goods again!"])

array([[ 0.3975833,  0.6024167]])

In [21]:
LR_pipeline.predict_proba(["I won't use any other vanilla!"])

array([[ 0.07592438,  0.92407562]])